# Project: Appointment Data Analysis

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> The Medical Appointments No Show dataset has been selected for this project. The dataset contains columns with information on identification of the patient, identification of each appointment, gender of the patient, day of the actual appointment, the day someone callled to make the appointment, the age of the patient, where the appointment takes place, scholarship under the Bolsa Familia, whether patient has hipertension, diabetes, alcoholism or handicap, number of messages sent to the patient and whether the patient showed up.
>
> Did patients who received an SMS have a higher rate of showing up? Does age affect the likelihood of a patient showing up? Which gender had a higher rate of showing up? Do patients with alcoholism have a higher chance of showing up?

In [55]:
# Importing all the packages needed for this project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling


In [56]:
# Read the dataset in from the website if it's not yet here

if not os.path.exists('appointments.csv'):
    !curl https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd2e9a_noshowappointments-kagglev2-may-2016/noshowappointments-kagglev2-may-2016.csv -o appointments.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.2M  100 10.2M    0     0  3450k      0  0:00:03  0:00:03 --:--:-- 3451k


In [57]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.
df = pd.read_csv('appointments.csv')
df.head(3)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No


### General Properties

In [58]:
df.shape

(110527, 14)

A simple observation of the dataset which we see has 110527 rows and 14 columns.

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


Checking to see if we have any missing values in out dataset. All columns have the same number of rows hence no missing values.

In [60]:
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


From the data above we can see that the mean age of the patients in this dataset is 37 years, the oldest being 115 years. 

The minimum age of a patient is -1 which looks like an error.

We can also see that majority of the patiens did not have hipertension, diabetes, alcoholism or handicap. 

Around 32% of the patients received SMS.

In [61]:
df.nunique()

PatientId          62299
AppointmentID     110527
Gender                 2
ScheduledDay      103549
AppointmentDay        27
Age                  104
Neighbourhood         81
Scholarship            2
Hipertension           2
Diabetes               2
Alcoholism             2
Handcap                5
SMS_received           2
No-show                2
dtype: int64

We can observe the unique values for each column. Columns with 2 are boolean and have either 1 for true/yes and 0 for false/no.

In [62]:
df.isnull().sum()

PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64

In [30]:
df.query('Gender == "M"').count()

Gender           38687
Age              38687
Neighbourhood    38687
Scholarship      38687
Hipertension     38687
Diabetes         38687
Alcoholism       38687
Handcap          38687
SMS_received     38687
No-show          38687
dtype: int64

In [63]:
df.dtypes

PatientId         float64
AppointmentID       int64
Gender             object
ScheduledDay       object
AppointmentDay     object
Age                 int64
Neighbourhood      object
Scholarship         int64
Hipertension        int64
Diabetes            int64
Alcoholism          int64
Handcap             int64
SMS_received        int64
No-show            object
dtype: object

We can see the data types for the columns and they are all convenient.

The dependent variable is No-show and the independent variables we will use in this analysis are SMS_received, Alcoholism, Scholarship, Age, and Gender.

We do not need PatientID, AppointmentID, ScheduledDay and AppointmentDay for this analysis.

## Data Cleaning 
### Dropping irrelevant columns and renaming the remaining ones

In [64]:
# Dropping irrelevant tables
df.drop(['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay'], axis=1, inplace=True)
df.head(1) # Confirm if columns have been dropped.


,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,F,62,JARDIM DA PENHA,0,1,0,0,0,0,No


Dropped the irrelevant tables.

In [65]:
idx = df.Age.idxmin()
idx

99832

In [66]:
df.loc[idx]

Gender               F
Age                 -1
Neighbourhood    ROMÃO
Scholarship          0
Hipertension         0
Diabetes             0
Alcoholism           0
Handcap              0
SMS_received         0
No-show             No
Name: 99832, dtype: object

Confirming the lowest Age is -1. We can drop it.

In [67]:
df.drop(labels=99832, inplace=True)

In [68]:
idx = df.Age.idxmin()
df.loc[idx] # Confirm if dropped

Gender                   F
Age                      0
Neighbourhood    CONQUISTA
Scholarship              0
Hipertension             0
Diabetes                 0
Alcoholism               0
Handcap                  0
SMS_received             0
No-show                 No
Name: 59, dtype: object

Age 0 is acceptable for babies

In [69]:
# Renaming columns for consistency
df.rename(columns=lambda x:x.strip().replace('-','_'),inplace=True)
df.columns.str.lower()

Index(['gender', 'age', 'neighbourhood', 'scholarship', 'hipertension',
       'diabetes', 'alcoholism', 'handcap', 'sms_received', 'no_show'],
      dtype='object')

This allows us to easily reference any column. The data is now clean we can begin the exploration phase.

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!